In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("NLP").getOrCreate()

data = spark.read.csv("SMSSpamCollection", inferSchema=True, sep="\t")

In [8]:
data = data.withColumnRenamed("_c0", "class").withColumnRenamed("_c1", "text")
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [12]:
from pyspark.sql.functions import length

data = data.withColumn("length", length(data["text"]))
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [14]:
data.groupBy("class").mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [15]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer

In [19]:
tokenizer = Tokenizer(inputCol="text", outputCol="token_text")
stop_remove = StopWordsRemover(inputCol="token_text", outputCol="stop_token")
count_vec = CountVectorizer(inputCol="stop_token", outputCol="c_vec")
idf = IDF(inputCol="c_vec", outputCol="tf_idf")
ham_spam_to_numeric = StringIndexer(inputCol="class", outputCol="label")

In [21]:
from pyspark.ml.feature import VectorAssembler
clean_up = VectorAssembler(inputCols=["tf_idf", "length"], outputCol="features")

In [22]:
from pyspark.ml.classification import NaiveBayes

nb = NaiveBayes()

In [24]:
from pyspark.ml import Pipeline

data_prep_pipe = Pipeline(stages=[ham_spam_to_numeric, tokenizer, stop_remove, count_vec, idf, clean_up])

In [25]:
cleaner = data_prep_pipe.fit(data)

Exception ignored in: <function JavaWrapper.__del__ at 0x0000023D8EC034C0>
Traceback (most recent call last):
  File "C:\Users\usuario\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pyspark\ml\wrapper.py", line 53, in __del__
    if SparkContext._active_spark_context and self._java_obj is not None:
                                              ^^^^^^^^^^^^^^
AttributeError: 'VectorAssembler' object has no attribute '_java_obj'


In [26]:
clean_data = cleaner.transform(data)

In [29]:
clean_data = clean_data.select("label", "features")
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,301,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,140...|
|  0.0|(13424,[10,53,102...|
|  0.0|(13424,[127,185,4...|
|  1.0|(13424,[1,47,121,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,117...|
|  1.0|(13424,[8,16,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,95,221...|
|  0.0|(13424,[555,1797,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,44...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [30]:
training, test = clean_data.randomSplit([0.7, 0.3])

In [32]:
spam_detector = nb.fit(training)
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [33]:
test_results = spam_detector.transform(test)

In [34]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,3,9,1...|[-568.91866512648...|[0.99999999999998...|       0.0|
|  0.0|(13424,[0,1,7,8,1...|[-1186.1500745430...|[1.0,5.4286545817...|       0.0|
|  0.0|(13424,[0,1,12,33...|[-441.65914773654...|[1.0,5.6236374106...|       0.0|
|  0.0|(13424,[0,1,14,18...|[-1376.2984931423...|[1.0,1.1249780541...|       0.0|
|  0.0|(13424,[0,1,14,79...|[-706.52882234053...|[1.0,1.8552054357...|       0.0|
|  0.0|(13424,[0,1,43,69...|[-616.96087869167...|[0.99922937771842...|       0.0|
|  0.0|(13424,[0,1,71,10...|[-702.40942411332...|[1.0,9.2885648877...|       0.0|
|  0.0|(13424,[0,1,150,1...|[-250.54276120913...|[0.57806942085535...|       0.0|
|  0.0|(13424,[0,1,881,1...|[-96.912423549207...|[0.99999998789127...|       0.0|
|  0.0|(13424,[0

In [36]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()

In [37]:
acc = acc_eval.evaluate(test_results)

In [38]:
print("ACC of NB Model")
print(acc)

ACC of NB Model
0.9250027355175389
